In [5]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset,DataLoader
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt

In [7]:
torch.manual_seed(123)

In [8]:
df = pd.read_csv('/content/fmnist_small.csv')
df.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,9,0,0,0,0,0,0,0,0,0,...,0,7,0,50,205,196,213,165,0,0
1,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,1,0,0,0,...,142,142,142,21,0,3,0,0,0,0
3,8,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,8,0,0,0,0,0,0,0,0,0,...,213,203,174,151,188,10,0,0,0,0


In [9]:
x = df.drop('label',axis=1)
y = df['label']

In [12]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state = 12)

In [13]:
x_train /= 255.0
x_test /= 255.0

In [14]:
class dataset(Dataset):

  def __init__(self, features, labels):

    self.features = torch.tensor(features, dtype = torch.float32)
    self.labels = torch.tensor(labels, dtype = torch.float32)

  def __len__(self):

    return len(self.features)

  def __getitem__(self, idx):

    return self.features[idx], self.labels[idx]

In [15]:
train_dataset = dataset(x_train.values, y_train.values)
test_dataset = dataset(x_test.values, y_test.values)

In [18]:
train_loader = DataLoader(train_dataset, batch_size = 64, shuffle = True)
test_loader = DataLoader(test_dataset, batch_size = 64, shuffle = False)

In [19]:
class Model(nn.Module):

  def __init__(self,num_features):
    super().__init__()

    self.model = nn.Sequential(
        nn.Linear(num_features, 128),
        nn.ReLU(),
        nn.Linear(128, 56),
        nn.ReLU(),
        nn.Linear(56, 10),
        )

  def forward(self,x):

    return self.model(x)



In [20]:
lr = 0.01
epochs = 10

In [21]:
model = Model(x_train.shape[1])
loss = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = lr)

In [23]:
for epoch in range(epochs):

  total_epoch_loss = 0

  for batch_features, batch_labels in train_loader:

    ypred = model(batch_features)
    batch_loss = loss(ypred, batch_labels.long())

    optimizer.zero_grad()
    batch_loss.backward()
    optimizer.step()

    total_epoch_loss += batch_loss.item()

  print(f'Epoch: {epoch+1}, Loss: {total_epoch_loss/len(train_loader)}')



Epoch: 1, Loss: 0.9404073504606882
Epoch: 2, Loss: 0.571957958539327
Epoch: 3, Loss: 0.5000616717338562
Epoch: 4, Loss: 0.43162097096443175
Epoch: 5, Loss: 0.43200469732284547
Epoch: 6, Loss: 0.4224578795830409
Epoch: 7, Loss: 0.38853144844373066
Epoch: 8, Loss: 0.384949104587237
Epoch: 9, Loss: 0.33174855132897696
Epoch: 10, Loss: 0.33279554545879364


In [26]:
model.eval()

Model(
  (model): Sequential(
    (0): Linear(in_features=784, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=56, bias=True)
    (3): ReLU()
    (4): Linear(in_features=56, out_features=10, bias=True)
  )
)

In [32]:
total = 0
correct = 0

with torch.no_grad():

  for batch_features, batch_labels in test_loader:

    ypred = model(batch_features)
    _, predicted = torch.max(ypred.data, 1)

    total = total + batch_labels.shape[0]
    correct = correct + (predicted == batch_labels).sum().item()

print(f'Accuracy: {100*correct/total}')

Accuracy: 83.16666666666667
